In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score, davies_bouldin_score

%matplotlib inline


# Carregando os dados
treino = pd.read_csv('data/train.csv')
teste = pd.read_csv('data/test.csv')

print("***** Treino *****")
print(treino.head())
print("\n")
print("***** Teste *****")
print(teste.head())
print("\n")
print("\n")
print("***** Treino *****")
print(treino.describe())
print("\n")
print("***** Teste *****")
print(teste.describe())

print("\n")
print("***** Colunas de Treino *****")
print(treino.columns.values)
print("\n")
print("\n")
# Campos vazios na lista de treino retornam o valor True abaixo
treino.isna().head()

# # Campos vazios na lista de teste retornam o valor True abaixo
teste.isna().head()

print("***** Quantidade de campos vazios no Treino *****")
print(treino.isna().sum())
print("\n")
print("***** Quantidade de campos vazios no Teste *****")
print(teste.isna().sum())

# Tratamento de valores faltantes
treino.fillna({'Age': treino['Age'].median(), 'Embarked': treino['Embarked'].mode()[0]}, inplace=True)
teste.fillna({'Age': teste['Age'].median(), 'Embarked': teste['Embarked'].mode()[0]}, inplace=True)

# Remoção de colunas não numéricas
colunas_para_remover = ['Name', 'Ticket', 'Cabin', 'Embarked','PassengerId']
treino.drop(colunas_para_remover, axis=1, inplace=True)
teste.drop(colunas_para_remover, axis=1, inplace=True)

# Convertendo variáveis categóricas (AQUI PRECISO QUE VC ANALISE)
labelEncoder = LabelEncoder()
treino['Sex'] = labelEncoder.fit_transform(treino['Sex'])
teste['Sex'] = labelEncoder.transform(teste['Sex'])

# Normalizando os dados
scaler = MinMaxScaler()
X = scaler.fit_transform(treino.drop(['Survived'], axis=1))
y = treino['Survived']

# Calculando a soma dos quadrados dentro dos clusters para diferentes números de clusters (AQUI PRECISO QUE VC ANALISE adpatei essa parte do site)
inertia = []
K_range = range(1, 12)
for k in K_range:
    #kmeans = KMeans(n_clusters=k, max_iter=600, algorithm='auto')
    kmeans = KMeans(n_clusters=k, max_iter=1000, n_init="auto",random_state=42)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

print("\n")
print("\n")
# Plotando o gráfico do cotovelo
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertia, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Inertia')
plt.title('Gráfico do Cotovelo para o Dataset Titanic')
plt.show()

# Aplicando K-Means
#kmeans = KMeans(n_clusters=2, max_iter=600, algorithm='auto')
kmeans = KMeans(n_clusters=k, max_iter=1000, n_init="auto",verbose=1)
kmeans.fit(X)

# Avaliando a correspondência entre os clusters e a sobrevivência
treino['Cluster'] = kmeans.labels_
correspondencia = pd.crosstab(treino['Survived'], treino['Cluster'])
print(correspondencia)

print("\n")
print("\n")
# Visualizações
# Distribuição de idade por cluster
sns.histplot(data=treino, x='Age', hue='Cluster', kde=True)
plt.title('Distribuição de Idade por Cluster')
plt.show()

# Distribuição de tarifa por sexo, colorida por cluster (AQUI ESTAVA COLOcando pra testar mas nao sei se faz sentido)
sns.scatterplot(data=treino, x='Fare', y='Sex', hue='Cluster')
plt.title('Tarifa por Sexo e Cluster')
plt.show()

# Relação entre Pclass e Sobrevivência
sns.barplot(x='Pclass', y='Survived', data=treino)
plt.title('Relação entre Classe e Sobrevivência')
plt.show()


Para avaliar métricas de cluster, consultar: https://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation

In [ ]:
#Obtendo os rótulos dos clusters
labels = kmeans.labels_

# Calculando o coeficiente de silhueta
silhouette = silhouette_score(X, labels)
print("Coeficiente de Silhueta:", silhouette)

# Calculando o índice de Davies-Bouldin
davies_bouldin = davies_bouldin_score(X, labels)
print("Índice de Davies-Bouldin:", davies_bouldin)

In [ ]:
treino.loc[treino.Cluster==0].describe()

In [ ]:
labelEncoder.inverse_transform([0, 1])

In [ ]:
treino.describe()

In [ ]:
sns.barplot(x='Cluster', y='Pclass',hue="Sex", data=treino)

In [ ]:
sns.barplot(x='Cluster', y='Survived',hue="Pclass", data=treino)